# Updating a proofreading table

This notebook contains functions and example scripts to update a proofreading table stored in an excel file

In [18]:
import os
import pandas as pd
import numpy as np
from fafbseg import flywire
from caveclient import CAVEclient

client = CAVEclient('flywire_fafb_production')

C:\Users\sebas\anaconda3\envs\EM\lib\site-packages\blessed\terminal.py:186: UserWarning: Failed to setupterm(kind='xterm-color'): Could not find terminal xterm-color
  warnings.warn(msg)


## A) Updating IDs

### 1. Loading the table

In [2]:
# Choose path and file
dataPath = r'C:\Users\sebas\Downloads'
fileName = 'All_Tm9_neurons_input_count_ME_R_20221103.xlsx'
filePath = os.path.join(dataPath,fileName)

In [3]:
#Loading file as DataFrame
df = pd.read_excel(filePath)
if df["seg_id"][0] == 'asdf': #Dropping the fisrt row ('asdf' was added as a walk-around to set that column values as type str)
    df = df.iloc[1: , :]
    df.reset_index(inplace=True,drop=True)
display(df.head(1))
segmentsIDs = df["seg_id"]
pre_IDs = df["presynaptic_ID"]
post_IDs = df["postsynaptic_ID"]

,presynaptic_ID,counts,count %,postsynaptic_ID,URL buhmann postsynapses (temporary fix for some links),URL buhmann postsynapses,Synapses proofread (Y/N),XYZ soma,XYZ,voxel_raw_x,...,name,seg_id,twigs proofread (Y/N),FlyWire proofread (Y/N),identified_in,lab authorship (Y/N),notes,annotations_link,Extra notes as comments,Working on
0,720575940659388801,38.0,28.571429,720575940627738884,https://ngl.flywire.ai/?json_url=https://globa...,https://ngl.flywire.ai/?json_url=https://globa...,true positive,no soma,"61065, 87199, 4730",61065.0,...,"Lamina monopolar cell 3, L3, [FBbt_00003721]",720575940659388801,Y,Y,"Fischbach & Dittrich, 1989",Y,NaN,https://ngl.flywire.ai/?json_url=https://globa...,NaN,NaN


### 2. Update

In [4]:
#Update IDs witth chunkedgraph module of CAVE

# For "segmentsIDs"
#Empty spaces are type float and will be filled with "0"
segmentsIDs_int = list(map(lambda x: 0 if type(x) == float else int(x),segmentsIDs)) # From str to int
#to create a np.zeros array is important for the next step
new_segmentsIDs_int = list(map(lambda x: np.zeros(1) if x == 0 else client.chunkedgraph.get_latest_roots(x),segmentsIDs_int))
#Updated IDs leading to more than one ID a single ID will be kept inside [] brakets.
new_segmentsIDs_str = list(map(lambda x: str(x[0]) if x.size == 1 else x,new_segmentsIDs_int)) # From int to str

In [5]:
# For "pre_IDs"
#Empty spaces are type float and will be filled with "0"
pre_IDs_int = list(map(lambda x: 0 if type(x) == float or x == 'INPUTS PROOFREAD'  else int(x),pre_IDs)) # From str to int
#to create a np.zeros array is important for the next step
new_pre_IDs_int = list(map(lambda x: np.zeros(1) if x == 0 else client.chunkedgraph.get_latest_roots(x),pre_IDs_int))
#Updated IDs leading to more than one ID a single ID will be kept inside [] brakets.
new_pre_IDs_str = list(map(lambda x: str(x[0]) if x.size == 1 else x,new_pre_IDs_int)) # From int to str

In [7]:
# For "post_IDs"
#Empty spaces are type float and will be filled with "0"
post_IDs_int = list(map(lambda x: 0 if type(x) == float else int(x),post_IDs)) # From str to int
#to create a np.zeros array is important for the next step
new_post_IDs_int = list(map(lambda x: np.zeros(1) if x == 0 else client.chunkedgraph.get_latest_roots(x),post_IDs_int))
#Updated IDs leading to more than one ID a single ID will be kept inside [] brakets.
new_post_IDs_str = list(map(lambda x: str(x[0]) if x.size == 1 else x,new_post_IDs_int)) # From int to str

### Seleting the right pre_ID if the update gaves more than one

In [124]:
#Getting the correct pre_IDs than contact each post_ID

from functools import reduce

correct_IDs = {}
curr_post_ID = 'Start'
for idx,pre_IDs in  enumerate(new_pre_IDs_str):
        
    #If there are multiple IDs in an array
    if type(pre_IDs) != str and type(pre_IDs) == np.ndarray:
        #Creatting synapses dataframe only once per each post_ID
        if curr_post_ID != new_post_IDs_str[idx]:
            synapses = flywire.synapses.fetch_synapses(new_post_IDs_str[idx], pre=False, post=True, attach=True,
                                          min_score=50, clean=True, transmitters=False,
                                          neuropils=True, batch_size=30,
                                          dataset='production', progress=True,mat="live")
            #Update post_ID
            print(f"Looking at post_ID: {new_post_IDs_str[idx]}") 
            curr_post_ID = new_post_IDs_str[idx]
            
        #Proof connectivity to the respective post_ID for each of them
        for ID in pre_IDs:
            if synapses[synapses['pre'] == ID].empty:
                continue
            else: # Only add the pre_ID (and its index) which has valid synapses with the post_IDs
                if idx in correct_IDs.keys(): # If there is already a valid ID, add other valid IDs 
                    curr_value =correct_IDs[idx]
                    new_value = curr_value+"_"+str(ID)
                    correct_IDs[idx] =new_value
                else:
                    correct_IDs[idx]=str(ID)

# Fixing the updated pre_IDs_str_list
for key, value in correct_IDs.items():
    new_pre_IDs_str[key] = value
    


Looking at post_ID: 720575940627738884
Looking at post_ID: 720575940633832287
Looking at post_ID: 720575940607562251
Looking at post_ID: 720575940612409571
Looking at post_ID: 720575940623950059
Looking at post_ID: 720575940618719915
Looking at post_ID: 720575940638092863


### Updating the dataframe

In [ ]:
# Adding the new url column to the data frame
df["Updated_pre_IDs"] = new_pre_IDs_str
df["Updated_post_IDs"] = new_post_IDs_str

### Saving back to excel file

In [139]:

# Creating string for the date
import datetime
x = datetime.datetime.now()
date_str = x.strftime("%d") + x.strftime("%b") + x.strftime("%Y")

# Writting in an existing excel file
from openpyxl import load_workbook
book = load_workbook(filePath)
writer = pd.ExcelWriter(filePath, engine = 'openpyxl')
writer.book = book

df.to_excel(writer, sheet_name='Updated_IDs_'+date_str)
writer.save()